# Dominik Adamczyk
## Laboratorium 4 - rozwiązania 

In [192]:
import numpy as np
import string
from collections import Counter
from itertools import combinations, combinations_with_replacement
import sklearn.cluster
from pandas import DataFrame

### N-grams

W zadaniu będę posługiwał się n-gramami dla metryk cosinusowej, Sorensena-Dice'a, euklidesowej.

In [40]:
def generate_ngrams(words, n):
    ngrams = [words[i:i+n] for i in range(len(words) - n + 1)]
    return Counter(ngrams)

## Metryki

1. Zaimplementuj przynajmniej 3 "metryki" spośród wymienionych: cosinusowa, LCS, DICE, euklidesowa, Levenshteina.

Wszystkie implementowane metryki będą w pewien sposób normalizowane, tak żeby teksty takie same po porównaniu metryką dawały wynik 0 (tożsame z odległością dwóch punktów na płaszczyźnie, które są w tym samym miejscu), a teksty całkowicie różne dawały w wyniku 1. Dzięki temu metryki będą w pewien sposób porównywalne do siebie.

### "Metryka" cosinusowa

$$
\begin{equation}
    \cos(\theta) = \frac{\mathbf{A} \cdot \mathbf{B}}{\|\mathbf{A}\| \|\mathbf{B}\|}= \frac{\sum\limits_{i=1}^{n} A_i B_i}{\sqrt{\sum\limits_{i=1}^{n} A_i^2} \sqrt{\sum\limits_{i=1}^{n} B_i^2}}
    \qquad\begin{aligned}
    &\text{gdzie:} \\
    &\mathbf{A}\text{ and }\mathbf{B} \text{ są porównywanymi wektorami}\\
    &n \text{ jest wymiarem wektora}\\
    &\theta \text{ reprezentuje kąt pomiędzy } \mathbf{A} \text{ i } \mathbf{B} \text{ wielowymiarowej przestrzeni}
    \end{aligned}
\end{equation}
$$

Wynik należy do przedziału $[-1,1]$. W tym zadaniu wynik będę odejmował od liczby 1 tak, aby takie same wektory miały odległość od siebie równą 0.


In [41]:
def cosine_similarity(A, B, n=2):
    n = min(len(A), len(B), n)
    A_grams = generate_ngrams(A, n)
    B_grams = generate_ngrams(B, n)
    if n == 0 : return 1
    return 1 - round(sum(A_grams[key] * B_grams[key] for key in A_grams.keys() & B_grams.keys()) / \
            (np.linalg.norm(list(A_grams.values())) * np.linalg.norm(list(B_grams.values()))), 15)


### Dystans euklidesowy 

$$
\begin{equation}
    d(x,y) = \sqrt{\sum_{i=1}^{n}(x_i-y_i)^2}
    \qquad\begin{aligned}
    &\text{gdzie:} \\
    &d(x,y) \text{ jest dystansem euklidesowym} \\
    &x_i, y_i \text{ są wartościami wektorów w i-tym wymiarze } x \text{ and } y \\
    &n \text{ jest wymiarem wektora}
    \end{aligned}
\end{equation}
$$

In [42]:
def euclidean_distance(A, B, n=2):
    n = min(len(A), len(B), n)
    A_grams = generate_ngrams(A, n)
    B_grams = generate_ngrams(B, n)
    if n == 0 : return 1
    keys = set(A_grams.keys()) | set(B_grams.keys())
    A_vals = np.array([A_grams[key] for key in keys])
    B_vals = np.array([B_grams[key] for key in keys])
    return np.linalg.norm(A_vals - B_vals) / np.linalg.norm(np.concatenate((A_vals, B_vals)))

### Współczynnik Dice

$$
\begin{equation}
    \text{Dice}(A, B) = \frac{2 |A \cap B|}{|A| + |B|} 
    \qquad\begin{aligned}
    &\text{gdzie:} \\
    &A \text{ and } B \text{ reprezentują dwa porównywane zbiory} \\
    &|A| \text{ i } |B| \text{ reprezentuje liczbę elementów zbioru} \\
    &\text{i } |A \cap B| \text{ reprezentuje liczbę elementów przecięcia zbiorów}
    \end{aligned}
\end{equation}
$$

In [43]:
def dice(A, B, n=2):
    n = min(len(A), len(B), n)
    if n==0: return 1
    A = set(generate_ngrams(A, n).keys())
    B = set(generate_ngrams(B, n).keys())
    
    return 1 - 2 * len(A & B) / (len(A) + len(B))
    

### Odległość Levenshteina

In [44]:
def levenshtein_distance(str1, str2):
    m = len(str1)
    n = len(str2)
    if n == 0 or m == 0: return 1
    dp = [[0 for j in range(n+1)] for i in range(m+1)]

    for i in range(m+1):
        dp[i][0] = i
    for j in range(n+1):
        dp[0][j] = j
    
    for i in range(1, m+1):
        for j in range(1, n+1):
            if str1[i-1] == str2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = 1 + min(dp[i-1][j], dp[i][j-1], dp[i-1][j-1])
    return dp[-1][-1] / max(m, n)

### LCS - Najdłuższy wspólny podciąg (Longest Common Subsequence)

In [45]:
def lcs(str1, str2):
    m = len(str1)
    n = len(str2)
    if n == 0 or m == 0: return 1
    lcs_table = [[0] * (n + 1) for i in range(m + 1)]
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if str1[i-1] == str2[j-1]:
                lcs_table[i][j] = lcs_table[i-1][j-1] + 1
            else:
                lcs_table[i][j] = max(lcs_table[i][j-1], lcs_table[i-1][j])

    return 1 - lcs_table[-1][-1] / max(m, n)

### Czy to w ogóle działa?

Poniżej sprawdzę czy metryki dają (w przybliżeniu) oczekiwane wyniki z przedziału $[0, 1]$. 

1. Teksty takie same

In [46]:
t1 = "Ala Ma kotA"
t2 = t1

print("Metryka cosinusowa: ", cosine_similarity(t1, t2))
print("Odległość euklidesowa: ", euclidean_distance(t1, t2))
print("Współczynnik Dice: ", dice(t1, t2))
print("Odległość levenshteina: ", levenshtein_distance(t1, t2))
print("Najdłuższy wspólny podciąg: ", lcs(t1, t2))


Metryka cosinusowa:  0.0
Odległość euklidesowa:  0.0
Współczynnik Dice:  0.0
Odległość levenshteina:  0.0
Najdłuższy wspólny podciąg:  0.0


2. Teksty zupełnie różne

In [47]:
t1 = "Ala Ma kotA"
t2 = "PieS_Nie_zJe"

print("Metryka cosinusowa: ", cosine_similarity(t1, t2))
print("Odległość euklidesowa: ", euclidean_distance(t1, t2))
print("Współczynnik Dice: ", dice(t1, t2))
print("Odległość levenshteina: ", levenshtein_distance(t1, t2))
print("Najdłuższy wspólny podciąg: ", lcs(t1, t2))


Metryka cosinusowa:  1.0
Odległość euklidesowa:  1.0
Współczynnik Dice:  1.0
Odległość levenshteina:  1.0
Najdłuższy wspólny podciąg:  1.0


3. Teksty bardzo podobne

In [48]:
t1 = "Ala ma mopsa"
t2 = "Ala ma psa"

print("Metryka cosinusowa: ", cosine_similarity(t1, t2))
print("Odległość euklidesowa: ", euclidean_distance(t1, t2))
print("Współczynnik Dice: ", dice(t1, t2))
print("Odległość levenshteina: ", levenshtein_distance(t1, t2))
print("Najdłuższy wspólny podciąg: ", lcs(t1, t2))


Metryka cosinusowa:  0.14365116142232504
Odległość euklidesowa:  0.3922322702763681
Współczynnik Dice:  0.17647058823529416
Odległość levenshteina:  0.16666666666666666
Najdłuższy wspólny podciąg:  0.16666666666666663


3. Teksty lekko podobne

In [49]:
t1 = "Ala ma mopsa"
t2 = "Owce lubią mopy"

print("Metryka cosinusowa: ", cosine_similarity(t1, t2))
print("Odległość euklidesowa: ", euclidean_distance(t1, t2))
print("Współczynnik Dice: ", dice(t1, t2))
print("Odległość levenshteina: ", levenshtein_distance(t1, t2))
print("Najdłuższy wspólny podciąg: ", lcs(t1, t2))


Metryka cosinusowa:  0.723973776263058
Odległość euklidesowa:  0.8509629433967631
Współczynnik Dice:  0.7391304347826086
Odległość levenshteina:  0.7333333333333333
Najdłuższy wspólny podciąg:  0.6666666666666667


## Ocena jakości klasteryzacji

2. Zaimplementuj przynajmniej 1 sposób oceny jakości klasteryzacji (np. indeks Daviesa-Bouldina)


W mojej implementacji centroidem będzie tekst w klastrze który ma najmniejszą odległość do wszystkich pozostałych tekstów według zadanej miary.
Decyzja spowodowana jest dopasowaniem algorytmu do wszystkich zaimplementowanych miar - w dwóch przypadkach tekst nie jest zamieniany na wektor częstości (dla LCS i Levenshteina)

### Indeks Daviesa-Bouldina

In [50]:
def get_centroid(cluster, metric):
    k = len(cluster)
    dist = np.zeros(k)
    for i, j in combinations(range(k), 2):
        to_add = metric(cluster[i], cluster[j])
        dist[i] += to_add
        dist[j] += to_add
    cluster = zip(dist, cluster)
    return min(cluster, key=lambda x : x[0])[1]
    

def intra_cluster_dispersion(cluster, metric):
    centroid = get_centroid(cluster, metric)
    return (sum([metric(text, centroid) ** 2 for text in cluster]) / len(cluster)) ** 0.5

def davies_bouldin_index(clusters, metric):
    n = len(clusters)
    centroids = [get_centroid(cluster, metric) for cluster in clusters]
    Si = [intra_cluster_dispersion(cluster, metric) for cluster in clusters]
    
    D_sum = 0
    
    for i in range(n):
        tmp = 0
        for j in range(n):
            if i == j: continue
            tmp = max(tmp, (Si[i] + Si[j]) / metric(centroids[i], centroids[j]))
        D_sum += tmp
    
    return D_sum / n

### Indeks Dunna

Definiuję wielkość klastra jako odległość między dwoma najodleglejszymi punktami w tym klastrze. Możliwości definicji jest więcej np. odległość wszystkich punktów danego klastra od ich centroidu.

Dystans pomiędzy klastrami zdefiniuję jako odległość między dwoma najbliższymi punktami należącymi do różnych klastrów.

Powyższe definicje są oryginalnymi dystansami zaproponowanymi przez Dunna.

In [51]:
def cluster_size(cluster, metric):
    return max(metric(ti, tj) for ti, tj in combinations_with_replacement(cluster, 2))

def intercluster_distance(cluster1, cluster2, metric):
    return min(metric(ti, tj) for ti in cluster1 for tj in cluster2)
    
def dunn_index(clusters, metric):
    numerator = min(intercluster_distance(c1, c2, metric) for (c1, c2) in combinations(clusters, 2))
    denominator = max(cluster_size(cluster, metric) for cluster in clusters)
    return numerator / denominator


### Czy to w ogóle działa?

Poniżej sprawdzę czy zaimplementowane przezemnie metody klasteryzacji dają sensowne wyniki. Dla indeksu Daviesa im lepsza klasteryzacja tym mniejszy wynik funkcji oceniającej, w przeciwieństwie do indeksu Dunna gdzie im lepsza klasteryzacja tym większy wynik.

1. Słaba klasteryzacja - teksty nie są za bardzo podobne do siebie

In [52]:
c = [["alamakot", "piesjemleko", "spaniefajnejest"],
     ["alkotomat", "mieszanie", "ciastko waniliowe"],
     ["lubie szarlotke", "ale jabłka muszą być dobre"]]
print("Davies-Bouldin index, lcs: ", davies_bouldin_index(c, lcs))
print("Dunn index, lcs: ", dunn_index(c,lcs))

print("Davies-Bouldin index, euclidean: ", davies_bouldin_index(c, euclidean_distance))
print("Dunn index, euclidean: ", dunn_index(c,euclidean_distance))

Davies-Bouldin index, lcs:  1.6588014978165446
Dunn index, lcs:  0.5555555555555555
Davies-Bouldin index, euclidean:  1.5393580236310427
Dunn index, euclidean:  0.6831300510639733


2. Lepsza klasteryzacja - teksty w klastrach są bardziej zbliżone do siebie

In [53]:
c = [["alamakot", "ala ma kota", "ala nie ma kota"],
     ["ciasto wanilinowe", "ciastko tiramisu waniliowe", "ciastko waniliowe"],
     ["lubie szarlotke", "szarlotka szarlotka"]]
print("Davies-Bouldin index, lcs: ", davies_bouldin_index(c, lcs))
print("Dunn index, lcs: ", dunn_index(c,lcs))

print("Davies-Bouldin index, euclidean: ", davies_bouldin_index(c, euclidean_distance))
print("Dunn index, euclidean: ", dunn_index(c,euclidean_distance))

Davies-Bouldin index, lcs:  0.8321434903110966
Dunn index, lcs:  1.1259259259259258
Davies-Bouldin index, euclidean:  0.899257071215224
Dunn index, euclidean:  1.1896993802573046


3. Słaba klasteryzacja - klastry są podobne do siebie

In [54]:
c = [["alamakot", "ala ma kota", "ala nie ma kota"],
     ["ala alkota ma", "ala kot", "kot ala"],
     ["alkomat", "ala z kotem"]]
print("Davies-Bouldin index, lcs: ", davies_bouldin_index(c, lcs))
print("Dunn index, lcs: ", dunn_index(c,lcs))

print("Davies-Bouldin index, euclidean: ", davies_bouldin_index(c, euclidean_distance))
print("Dunn index, euclidean: ", dunn_index(c,euclidean_distance))

Davies-Bouldin index, lcs:  2.480104415122002
Dunn index, lcs:  0.40625
Davies-Bouldin index, euclidean:  1.769539479896063
Dunn index, euclidean:  0.4364357804719848


4. Lepsza klasteryzacja - klastry nie są podobne do siebie

In [55]:
c = [["alamakot", "trwonić"],
     ["xxxxz", "zzzzzzz", "ssssss"],
     ["cccccc", "dddddd"]]
print("Davies-Bouldin index, lcs: ", davies_bouldin_index(c, lcs))
print("Dunn index, lcs: ", dunn_index(c,lcs))

print("Davies-Bouldin index, euclidean: ", davies_bouldin_index(c, euclidean_distance))
print("Dunn index, euclidean: ", dunn_index(c,euclidean_distance))

Davies-Bouldin index, lcs:  1.438059209240504
Dunn index, lcs:  1.0
Davies-Bouldin index, euclidean:  1.5236033621142735
Dunn index, euclidean:  1.0


Wygląda na to, że metody oceniania klasteryzacji działają poprawnie, a przynajmniej dają oczekiwane względne wyniki

## Stoplista
3. Stwórz stoplistę najczęściej występujących słów i zastosuj ją jako pre-processing dla nazw. Algorytmy klasteryzacji powinny działać na dwóch wariantach: z pre-processingiem i bez pre-processingu.


Znajduję częstość słów w klasteryzowanym tekści i zwracam pewien procent najpopularniejszych słów

In [164]:
def stoplist(text, frequency): # frequency from interval [0, 1]
    words = [word for line in text for word in line.split(' ')]
    words = list(Counter(words).items())
    words.sort(key=lambda x:x[1])
    # print(words)
    words = list(map(lambda x : x[0], words))
    
    # print(words)
    
    words = words[-int(frequency * len(words)):]
    # print(words)
    
    return words


### Preprocessing tekstu

Tekst przed użyciem w metrykach jest konwertowany na małe litery, a także usuwane są znaki interpunkcyjne (o ile będziemy tego chciali użyć w programie, ja chyba będę chciał).


In [165]:
def preprocess_punctuation(text):
    out = []
    for l in text:
        lowercase_text = l.lower()
    
        out.append(lowercase_text.translate(str.maketrans('', '', string.punctuation)))
    
    return out


In [166]:
example_text = ["The quIck brown $fox j:umPs OV@er the lazy !doG. 123456789.com"]
preprocess_punctuation(example_text)

['the quick brown fox jumps over the lazy dog 123456789com']

In [167]:
def preprocess_stoplist(text, frequency):
    to_remove = stoplist(text, frequency)
    out = []
    for l in text:
        out.append(" ".join([w for w in l.split(" ") if w not in to_remove]))
    return out

In [168]:
def preprocess_all(text, frequency):
    text = preprocess_punctuation(text)
    return preprocess_stoplist(text, frequency)
    

### Wczytanie tekstu

In [169]:
with open("lines.txt", "r", encoding="UTF-8") as f:
    text = f.read().split("\n")

for el in text[:10]: print(el)

/11692589 RD TUNA CANNERS, LTD. PORTION 1004, SIAR NORTH COAST ROAD, P.O.BOX 2113, MADANG, PAPUA NEW GUINEA
''PA INTERIOR'' LTD BOLSHAYA LUBYANKA STREET, 16/4 MOSCOW, 101000, RUSSIA INN/KPP 7704550148//770801001 495-984-8611
''SSONTEX''  Sp.ZO.O.IMPORT-EXPORTUL:PRZECLAWSKA 5 03-879 WARSZAWA,POLAND NIP 113-01-17-669
''SSONTEX''SP.ZO.O.IMPORT-EXPORT UL:PRZECLAWSKA 5 03-879 WARSZAWA,POLAND NIP 113-01-17-669 TEL./FAX.:0048(022)217 6532--
''TOPEX SP. Z O.O.'' SPOLKA KOMANDYTOWA UL. POGRANICZNA 2/4  02-285 WARSZAWA POLAND
'MASTER PLUS CO.,LTD.' 143000,RUSSIA,MO,ODINSOVO, MOJAISKOE, SHOSSE,153G TEL:+7495 7273939
"2TIGERS GROUP LIMITED"  ROOM 504 JINSHAZHOU SHANGSHUI ROAD,  GUANGZHOU 510160
"ALDETRANS" LLC, 105066, MOSCOW, RUSSIA, TOKMAKOV LANE, 11. TEL:+7(495)641-03-89
"A-LIFT",JSC 1 PROSPEKT MARSHALA ZHUKOVA,MOSCOW 123308,RUSSIA  T: +7(495)784-7961
"ALISA" LTD, 1/5 Derbenevskaya str., Moscow, Russia Tel./Fax: (495) 987-13-07 postal code: 115114


### Test preprocessingu

In [170]:
print(stoplist(text, 0.01))

['LTD,', '190020', 'SAME', 'NANJING', '6,', 'C/O', 'IKEA', 'Sp.', '32', '19', 'C', 'PRODUCTS', '424', '6', '66', 'KOTKA', 'PLAZA,', 'TOWN', 'GEODIS', 'LIT.', 'ST.PETERSBURG,', 'SDN', 'ZONE', 'LINES', '18', '3,', 'TOWER,', 'LOGISTIC', 'GERMANY', 'AIR', '660', '70', 'KUEHNE&NAGEL', 'SP.ZO.O.', 'ZONE,', 'EXPORT', 'SCAN', '39', 'SAMSUNG', '190020,', '82', 'EXPEDITION', 'ST.PETERSBURG', 'GDYNIA,POLAND', 'HANGZHOU', 'BENE', 'LIFLYANDSKAYA', 'CENTRE', 'TIANJIN', '83', 'ATTN:', 'INDIA', 'PANTOS', 'RUS', 'CN', 'TRANSPORT', 'CONTACT', 'ul.', 'PLAZA', 'SERVICES', 'KOREA', 'IMPORT&EXPORT', 'TRANS', 'PVT', '310', '37', '17', 'IMPORT', 'CENTER', 'JIANGSU', 'NAGEL', '621', 'O', 'SAVINO', 'STR.', '448', '13', 'OCEAN', 'KUEHNE', 'CO.', 'HONGKONG', '495', 'PARK', 'ELECTRONICS', 'PETERSBURG', '50', 'PHONE:', 'DEL', '12', 'SA', 'Poland', '2,', '21', 'SPOLKA', 'VIETNAM', 'DEVELOPMENT', 'TEL.', 'UL', 'SP.Z.O.O.', 'PANTAINER', '.', 'FEDERATION', 'AIR&SEA', '10', 'GDYNIA,', 'DSV', '31', '/', 'VANTAA', '44', '

usuwanie punktuacji

In [171]:
for el in preprocess_punctuation(text)[:10]: print(el)

11692589 rd tuna canners ltd portion 1004 siar north coast road pobox 2113 madang papua new guinea
pa interior ltd bolshaya lubyanka street 164 moscow 101000 russia innkpp 7704550148770801001 4959848611
ssontex  spzooimportexportulprzeclawska 5 03879 warszawapoland nip 1130117669
ssontexspzooimportexport ulprzeclawska 5 03879 warszawapoland nip 1130117669 telfax0048022217 6532
topex sp z oo spolka komandytowa ul pograniczna 24  02285 warszawa poland
master plus coltd 143000russiamoodinsovo mojaiskoe shosse153g tel7495 7273939
2tigers group limited  room 504 jinshazhou shangshui road  guangzhou 510160
aldetrans llc 105066 moscow russia tokmakov lane 11 tel74956410389
aliftjsc 1 prospekt marshala zhukovamoscow 123308russia  t 74957847961
alisa ltd 15 derbenevskaya str moscow russia telfax 495 9871307 postal code 115114


usuwanie ze stoplisty

In [172]:
for el in preprocess_stoplist(text, 0.01)[:10]: print(el)

/11692589 TUNA CANNERS, PORTION 1004, SIAR COAST P.O.BOX 2113, MADANG, PAPUA GUINEA
''PA INTERIOR'' BOLSHAYA LUBYANKA 16/4 101000, INN/KPP 7704550148//770801001 495-984-8611
''SSONTEX'' Sp.ZO.O.IMPORT-EXPORTUL:PRZECLAWSKA 03-879 WARSZAWA,POLAND NIP 113-01-17-669
''SSONTEX''SP.ZO.O.IMPORT-EXPORT UL:PRZECLAWSKA 03-879 WARSZAWA,POLAND NIP 113-01-17-669 TEL./FAX.:0048(022)217 6532--
''TOPEX O.O.'' KOMANDYTOWA POGRANICZNA 2/4 02-285
'MASTER PLUS CO.,LTD.' 143000,RUSSIA,MO,ODINSOVO, MOJAISKOE, SHOSSE,153G TEL:+7495 7273939
"2TIGERS LIMITED" 504 JINSHAZHOU SHANGSHUI 510160
"ALDETRANS" LLC, 105066, TOKMAKOV LANE, 11. TEL:+7(495)641-03-89
"A-LIFT",JSC PROSPEKT MARSHALA ZHUKOVA,MOSCOW 123308,RUSSIA T: +7(495)784-7961
"ALISA" 1/5 Derbenevskaya str., Moscow, Russia Tel./Fax: (495) 987-13-07 postal code: 115114


In [173]:
for el in preprocess_all(text, 0.01)[:10]: print(el)

11692589 tuna canners portion 1004 siar coast 2113 madang papua guinea
pa interior bolshaya lubyanka 164 101000 innkpp 7704550148770801001 4959848611
ssontex spzooimportexportulprzeclawska 03879 warszawapoland nip 1130117669
ssontexspzooimportexport ulprzeclawska 03879 warszawapoland nip 1130117669 telfax0048022217 6532
topex komandytowa pograniczna 24 02285
master plus 143000russiamoodinsovo mojaiskoe shosse153g tel7495 7273939
2tigers 504 jinshazhou shangshui 510160
aldetrans 105066 tokmakov lane tel74956410389
aliftjsc prospekt marshala zhukovamoscow 123308russia t 74957847961
alisa derbenevskaya 9871307 postal code 115114


### Klasteryzacja

4. Wykonaj klasteryzację zawartości załączonego pliku (lines.txt) przy użyciu  metryk zaimplementowanych w pkt. 1. Każda linia to adres pocztowy firmy, różne sposoby zapisu tego samego adresu powinny się znaleźć w jednym klastrze.

In [174]:
def clusterize(text, metric, eps=0.6, preprocess=True, frequency=0.02):
    if preprocess:
        text = preprocess_all(text, frequency)
    dist = [[metric(la, lb) for la in text] for lb in text]
    clust = sklearn.cluster.DBSCAN(eps=eps, min_samples=1).fit(dist)
    return clust.labels_

def text_clusters(text, metric, eps=0.6, preprocess=True, frequency=0.02):
    labels = clusterize(text, metric, eps, preprocess=preprocess, frequency=frequency)
    clusters = [[] for _ in range(max(labels) + 1)]
    for idx, cl in enumerate(labels):
        clusters[cl].append(text[idx])
    return clusters

def clusters_from_labels(text, labels):
    clusters = [[] for _ in range(max(labels) + 1)]
    for idx, cl in enumerate(labels):
        clusters[cl].append(text[idx])
    return clusters

## Testowanie
5. Porównaj jakość wyników sposobami zaimplementowanymi w pkt. 2.


Na początek spróbuję zwizualizować czy klasteryzacja działa.

Jako że algorytmy tutaj używane mają złożoność $O(n^2)$ - gdzie n to długość linijki (np. lcs) i są powtarzane po $O(m^2)$ razy, to sklasteryzowanie wszystkich linijek z pliku lines.txt zajmuje dużo czasu. Dlatego będę korzystać z pierwszych 150 linii załączonego pliku.

In [175]:
text = text[:150]
test_labels = clusterize(text, dice)
print(test_labels)


[  0   1   2   2   3   4   5   6   7   8   9  10  11  11  11  11  12  12
  12  13  13  14  15  15  16  17  18  19  19  20  21  22  23  23  24  25
  26  27  28  29  30  31  32  33  34  35  36  37  38  38  39  40  40  40
  41  42  41  42  43  43  43  43  43  43  44  45  46  47  48  48  49  50
  51  52  53  54  55  56  57  58  59  60  61  62  62  63  64  65   2  66
  67  68  69  70  71  72  73  73  73  73  73  74  75   3   3   3   3   3
  76   3   3  77   3   3  78  79  80  80  80  81  82  83  84  84  85  86
  87  88  89  90  91  92  93  94  95  96  97  98  99  99 100 101 102 102
 102 102 102 102 102 103]


In [176]:
test_clusters = text_clusters(text, dice)
for cluster in test_clusters[:30]:
    for line in cluster:
        print(line)
    print("\n" + "#"*10 + "\n")

/11692589 RD TUNA CANNERS, LTD. PORTION 1004, SIAR NORTH COAST ROAD, P.O.BOX 2113, MADANG, PAPUA NEW GUINEA

##########

''PA INTERIOR'' LTD BOLSHAYA LUBYANKA STREET, 16/4 MOSCOW, 101000, RUSSIA INN/KPP 7704550148//770801001 495-984-8611

##########

''SSONTEX''  Sp.ZO.O.IMPORT-EXPORTUL:PRZECLAWSKA 5 03-879 WARSZAWA,POLAND NIP 113-01-17-669
''SSONTEX''SP.ZO.O.IMPORT-EXPORT UL:PRZECLAWSKA 5 03-879 WARSZAWA,POLAND NIP 113-01-17-669 TEL./FAX.:0048(022)217 6532--
"SSONTEX" SP.ZO.O IMPORT-EXPORT 03-879 WARSZAWA UL PRZECLAWSKA 5 NIP:113-01-17-669

##########

''TOPEX SP. Z O.O.'' SPOLKA KOMANDYTOWA UL. POGRANICZNA 2/4  02-285 WARSZAWA POLAND
"TOPEX SP.Z.O.O."SP.K. UL.POGRANICZNA 2/4, 02-285 WARSZAWA.POLAND
"TOPEX SP.Z.O.O."SP.K. UL.POGRANICZNA 2/4,02-285 WARSZAWA POLAND
"TOPEX SP.Z O.O."SP.K. UL,POGRANICZNA 2/4 02-285 WARSZAWA
"TOPEX SP.Z O.O."SP.K. UL.POGRANICZNA 2/4,02--285 WARSZAWA
"TOPEX SP. Z O. O." SP. K. UL.POGRANICZNA 2/4,02-285 WARSZAWA,POLAND
"TOPEX SP.Z O.O."SP.K. UL.POGRANICZNA 2

#### Klasteryzacja rzeczywiście działa przy wybranej metryce, chociaż zdarzają się pojedyncze błędy

In [177]:
def get_correct_clusters(text):
    with open('clusters.txt', 'r') as f:
        clusters = list(f)
    # print(clusters)
    correct_clusters = []
    for line in text:
        line += "\n"
        c = 0
        for cline in clusters:
            if cline[0] == "#":
                c += 1
            elif cline == line:
                correct_clusters.append(c)
                break
    mappp = {}
    idx =0
    for i in correct_clusters:
        if i not in mappp.keys():
            mappp[i] = idx
            idx += 1
    out = []
    for el in correct_clusters:
        out.append(mappp[el])
    return np.array(out)

In [178]:
correct_labels = get_correct_clusters(text)
correct_labels

array([ 0,  1,  2,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 11, 11, 11, 11,
       11, 11, 12, 12, 13, 14, 14, 15, 16, 17, 18, 18, 19, 20, 21, 22, 22,
       23, 24, 25, 26, 27, 28, 29, 29, 30, 31, 32, 33, 34, 35, 36, 36, 37,
       38, 38, 38, 39, 39, 39, 39, 40, 40, 40, 40, 40, 40, 41, 41, 41, 42,
       42, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 56,
       57, 58, 59,  2, 60, 61, 62, 63, 64, 63, 64, 65, 65, 65, 65, 65, 66,
       67,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3, 68, 69, 70, 70, 70,
       71, 72, 73, 74, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86,
       87, 88, 89, 89, 90, 91, 27, 27, 26, 26, 26, 26, 26, 92])

In [179]:
correct_clusters = clusters_from_labels(text, correct_labels)
for cluster in correct_clusters[:30]:
    for line in cluster:
        print(line)
    print("\n" + "#"*10 + "\n")

/11692589 RD TUNA CANNERS, LTD. PORTION 1004, SIAR NORTH COAST ROAD, P.O.BOX 2113, MADANG, PAPUA NEW GUINEA

##########

''PA INTERIOR'' LTD BOLSHAYA LUBYANKA STREET, 16/4 MOSCOW, 101000, RUSSIA INN/KPP 7704550148//770801001 495-984-8611

##########

''SSONTEX''  Sp.ZO.O.IMPORT-EXPORTUL:PRZECLAWSKA 5 03-879 WARSZAWA,POLAND NIP 113-01-17-669
''SSONTEX''SP.ZO.O.IMPORT-EXPORT UL:PRZECLAWSKA 5 03-879 WARSZAWA,POLAND NIP 113-01-17-669 TEL./FAX.:0048(022)217 6532--
"SSONTEX" SP.ZO.O IMPORT-EXPORT 03-879 WARSZAWA UL PRZECLAWSKA 5 NIP:113-01-17-669

##########

''TOPEX SP. Z O.O.'' SPOLKA KOMANDYTOWA UL. POGRANICZNA 2/4  02-285 WARSZAWA POLAND
"TOPEX SP.Z.O.O."SP.K. UL.POGRANICZNA 2/4, 02-285 WARSZAWA.POLAND
"TOPEX SP.Z.O.O."SP.K. UL.POGRANICZNA 2/4,02-285 WARSZAWA POLAND
"TOPEX SP.Z O.O."SP.K. UL,POGRANICZNA 2/4 02-285 WARSZAWA
"TOPEX SP.Z O.O."SP.K. UL.POGRANICZNA 2/4,02--285 WARSZAWA
"TOPEX SP. Z O. O." SP. K. UL.POGRANICZNA 2/4,02-285 WARSZAWA,POLAND
"TOPEX SP.Z O.O."SP.K. UL.POGRANICZNA 2

## Ocena jakości klasteryzacji

Poniższe funkcje klasteryzują tekst

In [200]:
def test_clusterization(metric, to_cluster=text, correct_clusters=correct_clusters,  frequency=0.01):
    clusters_no_stoplist = text_clusters(to_cluster, metric)
    clusters_stoplist = text_clusters(to_cluster, metric, preprocess=True, frequency=frequency)

    return davies_bouldin_index(clusters_no_stoplist, metric), davies_bouldin_index(clusters_stoplist, metric), davies_bouldin_index(correct_clusters, metric), dunn_index(clusters_no_stoplist, metric), dunn_index(clusters_stoplist, metric), dunn_index(correct_clusters, metric)
    
def test_all():
    metrics = ['cosine_similarity', 'euclidean_distance', 'dice', 'levenshtein_distance', 'lsc']
    davies_nostop = []
    dunn_nostop = []
    davies_stop = []
    dunn_stop = []
    davies_corr = []
    dunn_corr = []
    for metric in [cosine_similarity, euclidean_distance, dice, levenshtein_distance, lcs]:
        dns, ds, dc, duns, dus, dcs = test_clusterization(metric)
        davies_nostop.append(dns)
        davies_stop.append(ds)
        davies_corr.append(dc)
        dunn_nostop.append(duns)
        dunn_stop.append(dus)
        dunn_corr.append(dcs)

    print("Davies bouldin index of clusterized dataset: ")
    df1 = DataFrame(({'metric':metrics, 'without stoplist': davies_nostop, 'with stoplist': davies_stop, 'clusters from file': davies_corr}))
    print(df1)
    print("\n\n")
    print("Dunn index of clusterized dataset: ")
    df2 = DataFrame(({'metric':metrics, 'without stoplist': dunn_nostop, 'with stoplist': dunn_stop, 'clusters from file': dunn_corr}))
    print(df2)


In [201]:
test_all()

Davies bouldin index of clusterized dataset: 
                 metric  without stoplist  with stoplist  clusters from file
0     cosine_similarity          0.653747       0.642862            0.878297
1    euclidean_distance          0.709269       0.706384            0.827266
2                  dice          0.690086       0.674226            0.834209
3  levenshtein_distance          0.551184       0.583951            0.719534
4                   lsc          0.632581       0.636623            0.780502



Dunn index of clusterized dataset: 
                 metric  without stoplist  with stoplist  clusters from file
0     cosine_similarity          0.058656       0.172505            0.072792
1    euclidean_distance          0.248138       0.353653            0.282142
2                  dice          0.069876       0.150278            0.085978
3  levenshtein_distance          0.067418       0.154253            0.170099
4                   lsc          0.069005       0.279603            

Indeks Daviesa osiąga mniejszą wartość dla lepszych klasteryzacji, a indeks Dunna większą wartość dla lepszych klasteryzacji.

Powyższe testy pokazują niewielką różnicę w jakości klasteryzacji na korzyść metod wykorzystujących stoplistę (ja użyłem stoplisty odpowiadającej 1% najbardziej popularnych słów). W niektórych przypadkach - dla indeksu Daviesa-Bouldina metryka Levenshteina i Lcs okazują się gorsze w metodzie ze stoplistą. Sytuacja ta nie uwidacznia się w indeksie Dunna. 

W każdym z przypadków najgorsza jakość klasteryzacji (według indeksów) jest osiągana przez zbiór zawierający wzorcową klasteryzację. Dzieje się tak, ponieważ implementowane metryki nie przedstawiają całkowicie rzeczywistego podobieństwa tekstów. Szukane klasteryzacje są najbardziej optymalne dla zadanych metryk, tekst rzeczywisty nie jest optymalizowany pod kątem metryki.

Według indeksu Daviesa-Bouldina do klasteryzacji najlepiej spisuje się odległość Levenshteina, lcs i podobieństwo cosinusowe. Indeks Dunna daje lepsze wyniki dla dystansu Euklidesowego, lcs i podobieństwa cosinusowego. Biorąc pod uwagę obydwie metody indeksacji najgorsze wyniki osiąga metryka DICE.

## Poprawa jakości klasteryzacji

6. Czy masz jakiś pomysł na poprawę jakości klasteryzacji w tym zadaniu?


* Lepszy preprocessing danych wejściowych - w tym zadaniu preprocessing nie został wykonany w najlepszy sposób, przez co niektóre wyrazy, które powinny występować oddzielnie zostały połączone

* Dobór parametrów modelu - zamiana parametrów takich jak eps może mieć wpływ na jakość klasteryzacji

* Wykorzystanie innych algorytmów - różne algorytmy klasteryzacji mają różne właściwości. W tym zadaniu użyto DBSCAN. Alternatywnimi algorytmami są np K-means, Gaussian Mixture Models

* Dobieranie odpowiednich metryk - metryka powinna być odpowiednio dostosowana do danych wejściowych

* Użycie sieci neuronowej. Można wytrenować odpowiednio mądry model który będzie sam potrafił zrobić taki clustering bez potrzeby używania algorytmów